In [1]:
# !pip3 install xgboost numpy textblob keras
from time import time
import pandas as pd
import numpy as np
import re
import csv
import matplotlib.pyplot as plt
import seaborn as sns

import itertools
import datetime

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier,AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn import decomposition, ensemble
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import metrics
# import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

In [ ]:
df = pd.read_csv('tweets_very_clean1.csv')

In [ ]:
twit = pd.read_csv('init_pagerank_twitter.csv', names=["user", "pr"])
twit

,user,pr
0,kucoincom,0.001044
1,BitCoin_Invest_,0.000573
2,KEEMSTAR,0.000537
3,pixelatedboat,0.000510
4,Boogie2988,0.000494
...,...,...
72824,Hextris5,0.000003
72825,Everycryptoday,0.000003
72826,6xsempre,0.000003
72827,JOBSAeK,0.000003


In [ ]:
df = df.merge(twit, on='user', how='left')

In [ ]:
vol = pd.read_csv('vol.csv')

In [ ]:
df = df.merge(vol, on='date', how='left')
df

,Unnamed: 0,index,date,id,user,fullname,url,timestamp,replies,likes,...,lang,year,month,day,text1,tags,promote,clean,pr,vol
0,0,0,2017-12-05,9.379548e+17,BisqArviraj,Ar Viraj,,2017-12-05 07:00:16+00,0.0,0.0,...,en,2017,12,5,SELL\nPayment method: SEPA\nOffer ID: 90196354...,,,sell payment method sepa offer id amount btc p...,0.000003,BULL
1,1,1,2017-12-05,9.379497e+17,cryptolligence,Cryptolligence,,2017-12-05 06:40:05+00,0.0,0.0,...,en,2017,12,5,Buy! (11:40:05 pm PDT)\nPrice: 11720.00 (+/-...,coinbase gdax btc bitcoin trading,,buy pm pdt price close stop gdax coinbase btc ...,0.000012,BULL
2,2,2,2017-12-05,9.379497e+17,cryptolligence,Cryptolligence,,2017-12-05 06:40:10+00,0.0,0.0,...,en,2017,12,5,Buy! (11:40:10 pm PDT)\nPrice: 11721.00 (+/-...,coinbase gdax btc bitcoin trading,,buy pm pdt price close stop gdax coinbase btc ...,0.000012,BULL
3,3,3,2017-12-05,9.379547e+17,coinstats,Coinstats,,2017-12-05 07:00:01+00,0.0,0.0,...,en,2017,12,5,"Bitcoin - BTC\nPrice: $11,861.80\nChange in 1h...",Bitcoin BTC,,bitcoin btc price change h market cap ranking ...,0.000036,BULL
4,4,4,2017-12-05,9.379536e+17,TNFXtrading,TNFX Trading,http://t.me/TNFXtrading,2017-12-05 06:55:38+00,0.0,0.0,...,en,2017,12,5,تنحصر تداولات سعر بتكوين الأخيرة داخل نموذج عل...,USDpic.twitter.com/oh8SKBx4DE BTC BTCUSD,,تنحصر تداولات سعر بتكوين الأخيرة داخل نموذج عل...,0.000018,BULL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332456,332456,1324380,2019-05-14,1.128253e+18,cointrend_jp,COINTREND,https://cointrend.jp/,2019-05-14 08:58:01+00,0.0,0.0,...,en,2019,5,14,ツイート数の多かった仮想通貨\n1位 $BTC 650 Tweets\n2位 $TRX 26...,,,ツイート数の多かった仮想通貨 位 btc tweets 位 trx tweets 位 xrp...,0.000093,BULL
332457,332457,1324381,2019-05-14,1.128249e+18,SssaPTAMUo5R7RE,Александр Свечкар,https://bi.city/s/KU0Gf,2019-05-14 08:41:43+00,0.0,0.0,...,en,2019,5,14,[BTC Dips Below 8000 USDT]\n\nCiting data from...,,,btc dips usdt citing data binance price bitcoi...,0.000003,BULL
332458,332458,1324382,2019-05-14,1.128251e+18,tr_ticker,Tr Ticker Bot,,2019-05-14 08:50:40+00,0.0,0.0,...,en,2019,5,14,#Doviz\n-------------------\n#USD : 6.1077\n#E...,Doviz BtcTurk BTC EUR Koinim Paribu GBP Gobaba...,,doviz usd eur gbp btc gobaba btcturk koinim pa...,0.000027,BULL
332459,332459,1324383,2019-05-14,1.128249e+18,EconomiesCom,Economies.com,http://dlvr.it/R4g63l,2019-05-14 08:41:02+00,0.0,0.0,...,en,2019,5,14,Midday update for bitcoin 14-05-2019: Bitcoin ...,,,midday update bitcoin bitcoin price resumes po...,0.000010,BULL


In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
def vader_compound(text):
    sid = SentimentIntensityAnalyzer()
    ss = sid.polarity_scores(text)
    compound, positive, negative, neutral = ss["compound"], ss["pos"], ss["neg"], ss["neu"]
    if compound > 0:
      return "BULL"
    return "BEAR"

In [ ]:
df['sent'] = df['clean'].apply(lambda x: vader_compound(str(x)))

In [ ]:
correct = 0
count = 0 
df = df.assign(comp=False)
same = []
for o, c in zip(df['sent'].iteritems(), df['vol'].iteritems()):
  index = o[0]
  op = o[1]
  cl = c[1]
  if op == cl:
    correct += 1
    df.loc[index,['comp']] = True
  count += 1
accuracy = float(correct) / float(count)
print(accuracy)

0.5116148961833117


In [ ]:
train_set = df[df["year"] != 2017]
test_set = df[df["year"] == 2017]
print(len(train_set))
print(len(test_set))

274811
57650


In [ ]:
train_set

,Unnamed: 0,index,date,id,user,fullname,url,timestamp,replies,likes,...,month,day,text1,tags,promote,clean,pr,vol,sent,comp
57650,57650,751439,2018-01-03,9.486873e+17,XRateMalaysia,Exchange-Rate.me,http://hargabitcoin.net,2018-01-03 21:47:34+00,0.0,0.0,...,1,3,"#Bitcoin 04 January 2018, 05:00 | USD: 15.077,...",Bitcoin,,bitcoin january usd idr,0.000003,BULL,BEAR,False
57651,57651,751440,2018-01-03,9.486873e+17,UOBGoldPrice,UOB Gold Price,http://hargabitcoin.net,2018-01-03 21:47:33+00,0.0,0.0,...,1,3,"#Bitcoin 04 January 2018, 05:00 | USD: 15.077,...",Bitcoin,,bitcoin january usd idr,0.000003,BULL,BEAR,False
57652,57652,751441,2018-01-03,9.486873e+17,XRateAU,XRateAU,http://hargabitcoin.net,2018-01-03 21:47:33+00,0.0,0.0,...,1,3,"#Bitcoin 04 January 2018, 05:00 | USD: 15.077,...",Bitcoin,,bitcoin january usd idr,0.000003,BULL,BEAR,False
57653,57653,751442,2018-01-03,9.486873e+17,XRateCA,XRateCA,http://hargabitcoin.net,2018-01-03 21:47:34+00,0.0,0.0,...,1,3,"#Bitcoin 04 January 2018, 05:00 | USD: 15.077,...",Bitcoin,,bitcoin january usd idr,0.000003,BULL,BEAR,False
57654,57654,751443,2018-01-03,9.486873e+17,XRateID,XRateID,http://hargabitcoin.net,2018-01-03 21:47:34+00,0.0,0.0,...,1,3,"#Bitcoin 04 January 2018, 05:00 | USD: 15.077,...",Bitcoin,,bitcoin january usd idr,0.000003,BULL,BEAR,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332456,332456,1324380,2019-05-14,1.128253e+18,cointrend_jp,COINTREND,https://cointrend.jp/,2019-05-14 08:58:01+00,0.0,0.0,...,5,14,ツイート数の多かった仮想通貨\n1位 $BTC 650 Tweets\n2位 $TRX 26...,,,ツイート数の多かった仮想通貨 位 btc tweets 位 trx tweets 位 xrp...,0.000093,BULL,BEAR,False
332457,332457,1324381,2019-05-14,1.128249e+18,SssaPTAMUo5R7RE,Александр Свечкар,https://bi.city/s/KU0Gf,2019-05-14 08:41:43+00,0.0,0.0,...,5,14,[BTC Dips Below 8000 USDT]\n\nCiting data from...,,,btc dips usdt citing data binance price bitcoi...,0.000003,BULL,BEAR,False
332458,332458,1324382,2019-05-14,1.128251e+18,tr_ticker,Tr Ticker Bot,,2019-05-14 08:50:40+00,0.0,0.0,...,5,14,#Doviz\n-------------------\n#USD : 6.1077\n#E...,Doviz BtcTurk BTC EUR Koinim Paribu GBP Gobaba...,,doviz usd eur gbp btc gobaba btcturk koinim pa...,0.000027,BULL,BEAR,False
332459,332459,1324383,2019-05-14,1.128249e+18,EconomiesCom,Economies.com,http://dlvr.it/R4g63l,2019-05-14 08:41:02+00,0.0,0.0,...,5,14,Midday update for bitcoin 14-05-2019: Bitcoin ...,,,midday update bitcoin bitcoin price resumes po...,0.000010,BULL,BULL,True


In [ ]:
train_rel_cols = train_set[["date", "user", "comp"]]
test_rel_cols = test_set[["date", "user", "comp"]]

In [ ]:
train_rel_cols["comp2"] = train_rel_cols["comp"]
train_agg = train_rel_cols.groupby("user").agg({"comp":"sum", "comp2":"count"})

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
len(train_agg)
train_agg.head()

,comp,comp2
user,,
000Bloger,1,1
000kkrish,2,2
002stefanie,1,1
002yt,0,1
0065Hiro,0,1


In [ ]:
matched_counts = train_agg["comp"].values.reshape(len(train_agg),1)
matched_counts_norm = matched_counts / np.max(matched_counts)
train_agg["matched_normalized"] = matched_counts_norm
train_agg.head(5)

,comp,comp2,matched_normalized
user,,,
000Bloger,1,1,0.000259
000kkrish,2,2,0.000518
002stefanie,1,1,0.000259
002yt,0,1,0.000000
0065Hiro,0,1,0.000000


In [ ]:
train_agg_sort = train_agg.sort_values(by="matched_normalized", ascending=False)

In [ ]:
train_agg_sort.to_csv("train_agg_with_score.csv")
train_set.to_csv("train_set.csv")
test_set.to_csv("test_set.csv")
df.to_csv("df.csv")

NameError: name 'train_agg_sort' is not defined

In [ ]:
train_pr_df = pd.DataFrame(df, columns=["user","pr"])

In [ ]:
train_pr_df

,user,pr
0,BisqArviraj,0.000003
1,cryptolligence,0.000012
2,cryptolligence,0.000012
3,coinstats,0.000036
4,TNFXtrading,0.000018
...,...,...
332456,cointrend_jp,0.000093
332457,SssaPTAMUo5R7RE,0.000003
332458,tr_ticker,0.000027
332459,EconomiesCom,0.000010


In [2]:
train_agg_sort = pd.read_csv('train_agg_with_score.csv')
train_set = pd.read_csv("train_set.csv")
test_set = pd.read_csv("test_set.csv")
df = pd.read_csv("df.csv")
train_pr_df = pd.DataFrame(df, columns=["user","pr"])

In [3]:
le = preprocessing.LabelEncoder()

In [ ]:
train_set.head()

,Unnamed: 0,Unnamed: 0.1,index,date,id,user,fullname,url,timestamp,replies,...,month,day,text1,tags,promote,clean,pr,vol,sent,comp
0,57650,57650,751439,2018-01-03,9.486873e+17,XRateMalaysia,Exchange-Rate.me,http://hargabitcoin.net,2018-01-03 21:47:34+00,0.0,...,1,3,"#Bitcoin 04 January 2018, 05:00 | USD: 15.077,...",Bitcoin,,bitcoin january usd idr,0.000003,BULL,BEAR,False
1,57651,57651,751440,2018-01-03,9.486873e+17,UOBGoldPrice,UOB Gold Price,http://hargabitcoin.net,2018-01-03 21:47:33+00,0.0,...,1,3,"#Bitcoin 04 January 2018, 05:00 | USD: 15.077,...",Bitcoin,,bitcoin january usd idr,0.000003,BULL,BEAR,False
2,57652,57652,751441,2018-01-03,9.486873e+17,XRateAU,XRateAU,http://hargabitcoin.net,2018-01-03 21:47:33+00,0.0,...,1,3,"#Bitcoin 04 January 2018, 05:00 | USD: 15.077,...",Bitcoin,,bitcoin january usd idr,0.000003,BULL,BEAR,False
3,57653,57653,751442,2018-01-03,9.486873e+17,XRateCA,XRateCA,http://hargabitcoin.net,2018-01-03 21:47:34+00,0.0,...,1,3,"#Bitcoin 04 January 2018, 05:00 | USD: 15.077,...",Bitcoin,,bitcoin january usd idr,0.000003,BULL,BEAR,False
4,57654,57654,751443,2018-01-03,9.486873e+17,XRateID,XRateID,http://hargabitcoin.net,2018-01-03 21:47:34+00,0.0,...,1,3,"#Bitcoin 04 January 2018, 05:00 | USD: 15.077,...",Bitcoin,,bitcoin january usd idr,0.000003,BULL,BEAR,False


In [ ]:
lr_data_train = train_set.merge(train_pr_df,on="user", how="left")
lr_data_train = lr_data_train.merge(train_agg_sort,on="user", how="left")
lr_data_train = lr_data_train.groupby("date").agg({"pr":"mean", "matched_normalized":"mean", "user":"count"})
lr_data_train = lr_data_train.rename(columns={"user":"count"})
lr_data_train = lr_data_train[["date", "pr", "matched_normalized", "count", "vol"]]
lr_data_train["vol"] = le.fit_transform(lr_data_train["vol"])

In [ ]:
lr_data_test = train_set.merge(train_pr_df,on="user", how="left")
lr_data_test = lr_data_test.merge(train_agg_sort,on="user", how="left")
lr_data_test = lr_data_test.groupby("date").agg({"pr":"mean", "matched_normalized":"mean", "user":"count"})
lr_data_test = lr_data_test.rename(columns={"username":"count"})
lr_data_test = lr_data_test[["date", "pr", "matched_normalized", "count", "vol"]]
lr_data_test["vol"] = le.fit_transform(lr_data_test["vol"])

In [ ]:
lr_train_Vals = lr_data_train[["pr", "matched_normalized"]].values
lr_train_target = lr_data_train[["vol"]].values

In [ ]:
weights = dict(Counter(lr_train_target.reshape(1,len(lr_train_target)).tolist()[0]))
total = sum(weights.values())
print(weights.keys())
for key_d in weights.keys():
    weights[key_d] /= float(total)
weights

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
clf = make_pipeline( StandardScaler(), SVC(gamma='scale',kernel="rbf"))
clf.fit(lr_train_Vals, np.ravel(lr_train_target))

In [ ]:
lr_test_Vals = lr_data_test[["pr", "matched_normalized"]].values
lr_test_target = lr_data_test[["vol"]].values

In [ ]:
preds = clf.predict(lr_test_Vals)
from sklearn.metrics import accuracy_score, f1_score,confusion_matrix
print("Accuracy: %.2f" % accuracy_score(np.ravel(lr_test_target), preds))
print("[tn, fp]")
print("[fn, tp]")
print(confusion_matrix(np.ravel(lr_test_target), preds))

In [ ]:
#ON THE MODEL THAT SHOWS MOST ACCURACY, DIG ON ITS SPECS
# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_predict_svm))
# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:",metrics.precision_score(y_test, y_predict_svm))
# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(y_test, y_predict_svm))

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=0147adb6-34ff-41f8-a148-e996b7669b04' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>